In [1]:
import os

In [2]:
os.getcwd()

'd:\\iNeuron\\Complete Project\\Deep-CNN_Classifier\\research'

In [3]:
os.chdir('../')

In [4]:
os.getcwd()

'd:\\iNeuron\\Complete Project\\Deep-CNN_Classifier'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir : Path
    trained_model_path: Path
    updated_base_model_path : Path
    training_data: Path
    params_epoch: int
    params_batch_size: int
    params_image_size: list
    params_is_augmentation: bool

In [7]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories

In [10]:
from pathlib import Path 

class ConfigurationManager:
    def __init__(self, 
                config_file_path = CONFIG_FILE_PATH, 
                params_file_path = PARAMS_FILE_PATH):
        try:
            self.config = read_yaml(config_file_path)
            self.params = read_yaml(params_file_path)
            create_directories([self.config.artifacts_root])
        except Exception as e:
            raise e

    def get_training_config(self):
        try:
            training = self.config.training
            prepare_base_model = self.config.prepare_base_model
            data_ingestion = self.config.data_ingestion
            params=self.params

            create_directories([Path(training.root_dir)])

            training_config = TrainingConfig(
                root_dir = Path(training.root_dir),
                trained_model_path=Path(training.trained_model_path),
                updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
                training_data=Path(os.path.join(data_ingestion.unzip_dir,"PetImages")),
                params_epoch=params.EPOCHS,
                params_batch_size=params.BATCH_SIZE,
                params_image_size=params.IMAGE_SIZE,
                params_is_augmentation=params.AUGMENTATION
            )
            return training_config
        except Exception as e:
            raise e
        
        

In [11]:
import tensorflow as tf


class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        try:
            self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
        except Exception as e:
            raise e
        
    def train_valid_generator(self):
        try:
            datagenerator_kwargs = dict(
                rescale = 1./255,
                vaidation_split = 0.2
            )

            dataflow_kwargs = dict(
                target_size = self.config.params_image_size,
                batch_size = self.config.params_batch_size,
                interpolation = "bilinear"
            )

            valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                **datagenerator_kwargs
            )

            self.valid_generator = valid_datagenerator.flow_from_directory(
                self.config.training_data,
                subset = "validation",
                shuffle = False,
                **dataflow_kwargs
            )

            if self.config.params_is_augmentation:
                training_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                    rotation_range = 40,
                    horizontal_flip = True,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    width_shift_range = 0.2,
                    height_shift_range = 0.2,
                    **datagenerator_kwargs
                )
            else:
                training_datagenerator = valid_datagenerator

            self.train_generator = training_datagenerator.flow_from_directory(
                    directory = self.config.training_data,
                    subset = "training",
                    shuffle = False,
                    **dataflow_kwargs
                )
        except Exception as e:
            raise e

    @staticmethod    
    def save_model(path: Path, model: tf.keras.Model):
        try:
            model.save(path)
        except Exception as e:
           raise e 
        
    def train(self):
        try:
            self.steps_per_epoch = self.train_generator.samples // self.config.params_batch_size
            self.validation_steps = self.valid_generator.samples // self.config.params_batch_size

            self.model.fit(
                self.train_generator,
                epochs = self.config.params_epoch,
                steps_per_epoch = self.steps_per_epoch,
                validation_steps = self.validation_steps,
                validation_data = self.valid_generator,
                callbacks = callback_list
            )

            self.save_model(
                path = self.config.trained_model_path,
                model  = self.model
            )
        except Exception as e:
            raise e

In [ ]:
try:
    config = ConfigurationManager()
    prepare_callbacks_model_config = config.get_prepare_callback_config()
    prepare_callbacks_model = PrepareCallbacks(config=prepare_callbacks_model_config)
    callback_list = prepare_callbacks_model.get_tb_ckpt_callbacks()

    #config = ConfigurationManager()
    training_config = config.get_training_config()
    training_model = Training(config=training_config)
    training_model.get_base_model()
    training_model.train_valid_generator()
    training_model.train(
        callback_list = callback_list
    )
except Exception as e:
    raise e 